In [1]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

DATA_PATH = "./resource"
# set seed
seed = 24
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pickle
import numpy as np

def save_pkl(path, obj):
  with open(path, 'wb') as f:
    pickle.dump(obj, f)
    print(" [*] save %s" % path)

def load_pkl(path):
  with open(path,'rb') as f:
    obj = pickle.load(f)
    print(" [*] load %s" % path)
    return obj

def save_npy(path, obj):
  np.save(path, obj)
  print(" [*] save %s" % path)

def load_npy(path):
  obj = np.load(path)
  print(" [*] load %s" % path)
  return obj

In [3]:
vocab = load_pkl(DATA_PATH + '/vocab.pkl')
TOTAL_NUM_CODES = len(vocab)
TOTAL_NUM_CODES

 [*] load ./resource/vocab.pkl


490

In [6]:
data = load_pkl(DATA_PATH + '/data.pkl')
label = load_pkl(DATA_PATH + '/label.pkl')

 [*] load ./resource/data.pkl
 [*] load ./resource/label.pkl


In [65]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(data,label,test_size=0.2,random_state=0)

In [72]:
X_train_flat = [item for sublist in X_train for item in sublist]
print(len(X_train_flat))
y_train_flat = [item for sublist in y_train for item in sublist]
print(len(y_train_flat))

191142
191142


In [73]:
from gensim.models import Word2Vec

# Dimensionality of the word vectors
VEC_SIZE = 100
# Ignores all words with total frequency lower than this
MIN_COUNT = 10

w2v_model = Word2Vec(X_train_flat, min_count=MIN_COUNT, vector_size=VEC_SIZE,workers=1)


In [84]:
X_train_embedded = [sum(w2v_model.wv[lst])/len(lst) if len(lst)>0 else np.zeros(VEC_SIZE) for lst in X_train_flat]

In [85]:
len(X_train_embedded)

191142

In [97]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(max_iter=200)
logreg.fit(X_train_embedded,y_train_flat)


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=200)

In [98]:
X_test_flat = [item for sublist in X_test for item in sublist]
print(len(X_test_flat))
y_test_flat = [item for sublist in y_test for item in sublist]
print(len(y_test_flat))
X_test_embedded = [sum(w2v_model.wv[lst])/len(lst) if len(lst)>0 else np.zeros(VEC_SIZE) for lst in X_test_flat]
y_pred=logreg.predict(X_test_embedded)
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test_flat, y_pred)
cnf_matrix

48794
48794


array([[35951,  1162],
       [ 8783,  2898]])

In [99]:
print("Accuracy:",metrics.accuracy_score(y_test_flat, y_pred))
print("Precision:",metrics.precision_score(y_test_flat, y_pred))
print("Recall:",metrics.recall_score(y_test_flat, y_pred))
print("F1:",metrics.f1_score(y_test_flat, y_pred))


Accuracy: 0.7961839570438989
Precision: 0.7137931034482758
Recall: 0.2480951973289958
F1: 0.368210405946255


In [100]:
y_pred_proba = logreg.predict_proba(X_test_embedded)[::,1]
auc = metrics.roc_auc_score(y_test_flat, y_pred_proba)
auc

0.7588047358323263